# ===============================
# MODE COLLAPSE TEST NOTEBOOK
# ===============================

In [ ]:
# ✅ Install Dependencies
!pip install -q transformers datasets nltk sentence-transformers tqdm pandas numpy

In [ ]:
import nltk
nltk.download('punkt', quiet=True)

from transformers import pipeline
from datasets import load_dataset
from nltk import ngrams
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np
import pandas as pd

## 1️⃣ Load Dataset (Synthetic Persona-Chat)

In [ ]:
print("📦 Loading dataset...")
try:
    ds = load_dataset("google/Synthetic-Persona-Chat", split="train")
    df = pd.DataFrame(ds)
except Exception as e:
    print("⚠️ Couldn't load from hub:", e)
    print("Trying manual CSV load instead...")
    df = pd.read_csv("hf://datasets/google/Synthetic-Persona-Chat/data/Synthetic-Persona-Chat_train.csv")

print("\n✅ Dataset loaded successfully!")
print("Available columns:", df.columns.tolist())
print(df.head(2))

# Automatically detect the most likely dialogue column
dialogue_col = None
for c in df.columns:
    if any(k in c.lower() for k in ["dialog", "conversation", "prompt", "text", "utterance"]):
        dialogue_col = c
        break

if not dialogue_col:
    raise ValueError("❌ No suitable dialogue column found in dataset. Check column names above.")

# Use a subset for testing
prompts = df[dialogue_col].astype(str).head(100).tolist()
print(f"\nUsing column '{dialogue_col}' with {len(prompts)} prompts.")

## 2️⃣ Load Models

In [ ]:
print("\n🧠 Loading text-generation models...")
model_names = {
    "GPT-2": "gpt2",
    "DistilGPT-2": "distilgpt2"
}
models = {}

for name, model_id in model_names.items():
    try:
        print(f"→ Loading {name}...")
        models[name] = pipeline("text-generation", model=model_id, device_map="auto")
    except Exception as e:
        print(f"⚠️ Failed to load {name}: {e}")

## 3️⃣ Generate Outputs

In [ ]:
def generate_responses(model, prompts, max_new_tokens=40):
    responses = []
    for p in tqdm(prompts, desc="🔁 Generating", leave=False):
        try:
            out = model(p, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.8)
            responses.append(out[0]["generated_text"])
        except Exception as e:
            responses.append("")
    return responses

responses_dict = {}
for name, model in models.items():
    print(f"\n🗣 Generating responses with {name}...")
    responses_dict[name] = generate_responses(model, prompts)

## 4️⃣ Diversity Metrics

In [ ]:
def distinct_n(texts, n):
    all_ngrams = [ng for t in texts for ng in ngrams(t.split(), n)]
    if not all_ngrams:
        return 0.0
    return len(set(all_ngrams)) / len(all_ngrams)

def self_bleu(texts, sample_size=30):
    sample_texts = texts[:sample_size]
    smoothie = SmoothingFunction().method1
    scores = []
    for i in range(len(sample_texts)):
        refs = [t.split() for j, t in enumerate(sample_texts) if j != i]
        if len(refs) == 0: continue
        score = sentence_bleu(refs, sample_texts[i].split(), smoothing_function=smoothie)
        scores.append(score)
    return np.mean(scores) if scores else 0.0

print("\n📊 Calculating diversity metrics...")

metrics = {}
for name, responses in responses_dict.items():
    d1 = distinct_n(responses, 1)
    d2 = distinct_n(responses, 2)
    sb = self_bleu(responses)
    metrics[name] = {"Distinct-1": d1, "Distinct-2": d2, "Self-BLEU": sb}
    print(f"\n{name} Metrics:")
    print(f"Distinct-1: {d1:.4f}, Distinct-2: {d2:.4f}, Self-BLEU: {sb:.4f}")

## 5️⃣ Embedding Variance (Semantic Diversity)

In [ ]:
print("\n🧩 Computing embedding variance (semantic diversity)...")
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def embedding_variance(texts):
    embeddings = embedder.encode(texts, show_progress_bar=False)
    return np.mean(np.var(embeddings, axis=0))

for name, responses in responses_dict.items():
    var = embedding_variance(responses)
    metrics[name]["EmbeddingVariance"] = var
    print(f"{name} Embedding Variance: {var:.6f}")

## 6️⃣ Results & Mode Collapse Analysis

In [ ]:
print("\n==================== MODE COLLAPSE ANALYSIS ====================")
print("Indicators of mode collapse:")
print("- High Self-BLEU → Less diversity")
print("- Low Distinct-n → Repetitive wording")
print("- Low Embedding Variance → Semantic similarity\n")

for name, vals in metrics.items():
    print(f"\n{name}:")
    for k, v in vals.items():
        print(f"  {k}: {v:.4f}")

# Simple thresholds for interpretation
def detect_collapse(m):
    return m["Self-BLEU"] > 0.8 or m["Distinct-1"] < 0.05 or m["EmbeddingVariance"] < 0.005

print("\n==================== CONCLUSION ====================")
collapse_flags = {name: detect_collapse(vals) for name, vals in metrics.items()}
for name, flag in collapse_flags.items():
    if flag:
        print(f"⚠️ {name}: Possible mode collapse detected.")
    else:
        print(f"✅ {name}: No strong signs of mode collapse.")

## 7️⃣ Summary Table

In [ ]:
print("\n📄 Summary Table:")
summary = pd.DataFrame(metrics).T
summary